# HW 2: Exploring JSON Files (Spring 2023)
----------------------------------
**Dr. Dave Wanik - OPIM 5502: Big Data Analytics with Cloud Computing - University of Connecticut**

The purpose of this assignment is to get you comfortable working with JSON files. Let's keep the scope small and focus on two JSON files, and if you want to go out on your own and easily scrape 500 free search engine result pages (SERP), then check out the appendix for SERP API!

* Your name here: Urvashi Bhurase
* Your NetID here: 3059409

Q1 and Q11 are worth 5 points, every other question is worth 10 points. Good luck!

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Download Data

In [ ]:
!gdown 1ejm645rRkuIdt1VETpICmJbLGyxMCf7F
!gdown 1nGfw8MOq_gYV55uJuFQ_1wgjpSIsvVw1

Downloading...
From: https://drive.google.com/uc?id=1ejm645rRkuIdt1VETpICmJbLGyxMCf7F
To: /content/drive/MyDrive/Colab_Notebooks/UCONN_Spring_2023/OPIM_5502_Big_Data_Analytics_with_Cloud_Computing/Module_3/Ukraine.json
100% 33.5k/33.5k [00:00<00:00, 36.9MB/s]
Downloading...
From: https://drive.google.com/uc?id=1nGfw8MOq_gYV55uJuFQ_1wgjpSIsvVw1
To: /content/drive/MyDrive/Colab_Notebooks/UCONN_Spring_2023/OPIM_5502_Big_Data_Analytics_with_Cloud_Computing/Module_3/London.json
100% 30.6k/30.6k [00:00<00:00, 43.7MB/s]


# Install Spark

In [2]:
# install java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# install spark (change the version number if needed)
# note that this is grabbing from the archive
!wget -q https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz

# unzip the spark file to the current folder
!tar xf spark-3.0.0-bin-hadoop3.2.tgz

# set your spark folder to your system path environment. 
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop3.2"


# install findspark using pip
!pip install -q findspark

# findspark
import findspark
findspark.init()

# Start session

We start the builder pattern `SparkSession.builder` and then chain a configuration parameter that defined the application name.

Providing a useful `appName` helps you identify which programs are running on your Spark cluster.

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder\
        .appName("Exploring_JSON_actors")\
        .getOrCreate()

In [ ]:
import pyspark.sql.functions as F

# Q1.  Inspect the .html and .json files

Imagine that someone just sent you the search results for these two queries. You can see that someone has been searching for 'top actors' in various cities (Kyiv, Ukraine and London, England.) The first geographic name referes to the 'top actors', and while the second set of geographic names looks duplicative, it is actually where you are telling Google to search from! 

If you dig through the two JSON files, you can actually click through and see the HTML result! It is buried in the JSON file but you can find it after looking around. Just lop off the **%7C** at the end and you can see the raw webpage you grabbed.
* https://serpapi.com/searches/2667d9f1ec89c315/63f12abf33b236a2bb4e555d.html%7C

Here are the clean HTML files:
* https://serpapi.com/searches/0fb523f94d3aeeac/63f12abda3f4ef6286fc8dfd.html
* https://serpapi.com/searches/2667d9f1ec89c315/63f12abf33b236a2bb4e555d.html

You can also replace the .html and just get the .json right away, too.
* https://serpapi.com/searches/0fb523f94d3aeeac/63f12abda3f4ef6286fc8dfd.json
* https://serpapi.com/searches/2667d9f1ec89c315/63f12abf33b236a2bb4e555d.json


Now that you have examined both the .html and .json files for these two search queries, **write down three interesting observations that you see**. Pay attention to how the structure of the .html website is efficiently captured in the .json file. No code here - just your qualitative observations.

## Interesting Thing 1

The json related basic information like exact search query, browser used, type of system the search was made is recorded by default under search_parameters.


## Interesting Thing 2

The data is recorded in the format of nested dictionary, where the value corresponding to a particular key of the outer dictionary can again be a dictionary or a list depending on the schema.

## Interesting Thing 3

For a google search query, majority of the query related information is recorded under search_information key of outer main dictionary and most of the keys of outer dictionary are common/default (for example, search_metadata, search_parameters, search_information, serpapi_pagination etc.) along with some additional keys(like inline_images for Ukraine.json and related_questions in London.json ) depending on the display result of the query.


# 🔴 Ukraine

I think this is the easier file to work with, so let's start here.

# Q2. Read the Ukraine .json file and printSchema()

Describe what you see in the schema.

In [ ]:
%ls -l /content

total 219276
drwx------  6 root root      4096 Mar 10 21:29 drive/
-rw-r--r--  1 root root     30588 Mar 10 18:16 London.json
drwxr-xr-x  1 root root      4096 Mar  9 18:58 sample_data/
drwxr-xr-x 13 1000 1000      4096 Jun  6  2020 spark-3.0.0-bin-hadoop3.2/
-rw-r--r--  1 root root 224453229 Jun  6  2020 spark-3.0.0-bin-hadoop3.2.tgz
-rw-r--r--  1 root root     33457 Mar 10 18:16 Ukraine.json


In [ ]:
ukraine_df = spark.read.json("/content/Ukraine.json")

In [ ]:
ukraine_df.printSchema()

root
 |-- inline_images: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- link: string (nullable = true)
 |    |    |-- original: string (nullable = true)
 |    |    |-- source: string (nullable = true)
 |    |    |-- source_name: string (nullable = true)
 |    |    |-- thumbnail: string (nullable = true)
 |    |    |-- title: string (nullable = true)
 |-- organic_results: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- about_page_link: string (nullable = true)
 |    |    |-- about_page_serpapi_link: string (nullable = true)
 |    |    |-- about_this_result: struct (nullable = true)
 |    |    |    |-- source: struct (nullable = true)
 |    |    |    |    |-- description: string (nullable = true)
 |    |    |    |    |-- icon: string (nullable = true)
 |    |    |    |    |-- security: string (nullable = true)
 |    |    |    |    |-- source_info_link: string (nullable = true)
 |    |    |-- cached_page_link: 

In Ukraine.json I see some basic default keys of json like search meta data, search parameters, search pagination, search information. Apart from that organic results outer dictionary key has an element struct which inturn has struct, array, string elements inside it.

# Q3. Extract the `organic_results.snippet` from the Ukraine .json
Are there any actors listed here? What does the output look like?

I have displayed the output in three different styles that I can come up with. In my opinion the third chunk is more preferable as it displays information in a more wholesome format.

In [ ]:
ukraine_df.select(F.explode('organic_results.snippet')).show(1,False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------+
|col                                                                                                                                                      |
+---------------------------------------------------------------------------------------------------------------------------------------------------------+
|Alec Utgoff is a British actor best known for playing 'Alexei' in the Netflix hit show, ... Ivanna Sakhno was born on November 14, 1997 in Kiev, Ukraine.|
+---------------------------------------------------------------------------------------------------------------------------------------------------------+
only showing top 1 row



In [ ]:
ukraine_df.select("organic_results.snippet").show(1, False)


+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
ukraine_df.select(F.explode("organic_results.snippet").alias("snippet")).show(8, False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------+
|snippet                                                                                                                                                  |
+---------------------------------------------------------------------------------------------------------------------------------------------------------+
|Alec Utgoff is a British actor best known for playing 'Alexei' in the Netflix hit show, ... Ivanna Sakhno was born on November 14, 1997 in Kiev, Ukraine.|
|From Ben Stiller to Jessica Chastain, celebrities have embraced Ukraine's president and offered support to the country's war effort.                     |
|Zelenskyy wrote on his social media page that Sean Penn is in Ukraine filming a movie about the war. He also said that Penn was in Ukraine ...           |
|Ukrainian actor Roman Matsiyta was ready to fight once the war 

I don't think we get actors list in the output, but actor's names present are present in each text description in the output. The output is the description mentioned under each website listed from the query result(this can be verified looking at the html).

# Q4. Extract the `organic_results.snippet_highlighted_words` from the Ukraine .json
Are there any actors listed here? What does the output look like?

In [ ]:
highlighted_words = ukraine_df.select(F.explode('organic_results.snippet_highlighted_words').alias("highlighted_words"))
highlighted_words.show(8, False)


+------------------------------+
|highlighted_words             |
+------------------------------+
|[actor best, Kiev, Ukraine]   |
|[Ukraine's]                   |
|null                          |
|[Ukrainian actor]             |
|[Ukrainian, actor]            |
|[actor, city, Ukrainian, Kyiv]|
|[Top Kyiv, Kyiv, Ukraine]     |
|[city, Kyiv, Ukrainian, city] |
+------------------------------+



In [ ]:
highlighted_words = ukraine_df.select("organic_results.snippet_highlighted_words")
highlighted_words.show(1, False)

+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|snippet_highlighted_words                                                                                                                                                   |
+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|[[actor best, Kiev, Ukraine], [Ukraine's],, [Ukrainian actor], [Ukrainian, actor], [actor, city, Ukrainian, Kyiv], [Top Kyiv, Kyiv, Ukraine], [city, Kyiv, Ukrainian, city]]|
+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------+



No there are no actors listed here. The output contains intersecting words present in the search query and the search output.

# Q5. Extract the `search_parameters.q` to get the name of the search query
Does the search you extracted match the search on the HTML page?



In [ ]:
search_parameters_subset = ukraine_df.select("search_parameters.q")
search_parameters_subset.show(1, False)

+------------------------------------+
|q                                   |
+------------------------------------+
|top actors in Kyiv,Kyiv city,Ukraine|
+------------------------------------+



Yes indeed the search extracted match the search on the HTML page.

# Q6. Extract the list of 12 names listed as the first SERP result from the Ukraine .json
Where was this information hiding in the .json?

In [ ]:
SERP_result = ukraine_df.select("organic_results.rich_snippet_list")
SERP_result.show(1, False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|rich_snippet_list                                                                                                                                                                            |
+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|[[Ivanna Sakhno, Natalie Burn, Gene Stupnitsky, Ilia Volok, Oleg Zagorodnii, Aleksey Gorbunov, Ana Layevska, Larisa Polonsky, Anna Sten, Vadim Perelman, Anna Sedokova, Alex Feldman],,,,,,,]|
+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+



The information was hidden on path organic_results-->element-->rich_snippet_list.

# Q7. Extract a list of the 9 websites listed on the Ukraine .json
Sometimes Google (SERP API) does not return 10 related searches, it will give you less! To be clear, I would like all of the links from the 9 positions within the organic results. Names should include imdb.com, theguardian.com, euronews.com etc.

In [ ]:
ukraine_websites = ukraine_df.select(F.col("organic_results.displayed_link"))
ukraine_websites.printSchema()

root
 |-- displayed_link: array (nullable = true)
 |    |-- element: string (containsNull = true)



In [ ]:
ukraine_websites.select(F.explode("displayed_link").alias("Ukraine_Search_Websites")).show(9, False)

+------------------------------------------------------------+
|Ukraine_Search_Websites                                     |
+------------------------------------------------------------+
|https://www.imdb.com › search › name › birth_place=...      |
|https://www.theguardian.com › world › jan › ukraine-h...    |
|https://www.euronews.com › video › 2022/06/28 › us-act...   |
|https://nypost.com › 2022/03/06 › ukrainian-actor-who...    |
|https://www.newsweek.com › ... › United Nations             |
|https://www.tribuneindia.com › Nation                       |
|https://www.tripadvisor.com › ... › Theaters in Kyiv        |
|https://www.latimes.com › business › story › actor-sean-p...|
+------------------------------------------------------------+



I don't see 9 websites listed in Ukraine.json and in the html. I guess you counted an extra imdb website that comes under organic_results-->related_results-->displayed_link
My code returns websites under  organic_results-->displayed_link. Let me know if I am missing something here!! To the best of my knowledge this is the correct output. Unless you want me to display the imdb link twice.


# 🔴 London

# Q8. Extract the list of famous actors from the first result on the London .json

```
Richard Foreman. Christian Bale. ...
Nick Briggs. Sean Bean. ...
Kate Beckinsale. ...
Dirk Bogarde. ...
Michael Caine. ...
John Cleese. ...
Sacha Baron Cohen. ..
```

This one is a bit tough to extract because it is 'hiding' in an answer box (notice how this first search result is prominent and specially formatted by Google.) It is NOT the first result (from Position 1) of the organic results.

While it is easy for you to just retrieve the answer (once you find it in the JSON), why not practice your PySpark and also clean up the information - I see 9 names in the answer box, please make me a table with one column called actors and 9 rows, one for each actor. **Hint:** You can use the `.` as a delimiter and replace all `...` with nothing...

In [ ]:
london_df = spark.read.json("/content/London.json")

In [ ]:
london_df.printSchema()

root
 |-- answer_box: struct (nullable = true)
 |    |-- about_this_result: struct (nullable = true)
 |    |    |-- source: struct (nullable = true)
 |    |    |    |-- description: string (nullable = true)
 |    |    |    |-- icon: string (nullable = true)
 |    |    |    |-- security: string (nullable = true)
 |    |    |    |-- source_info_link: string (nullable = true)
 |    |-- date: string (nullable = true)
 |    |-- displayed_link: string (nullable = true)
 |    |-- images: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- link: string (nullable = true)
 |    |-- list: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- snippet: string (nullable = true)
 |    |-- snippet_highlighted_words: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- title: string (nullable = true)
 |    |-- type: string (nullable = true)
 |-- organic_results: array (nullable = true)
 |    |-- elemen

In [ ]:
actor_list = london_df.select("answer_box.list")
actor_list.show(1, False)


+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|list                                                                                                                                                                                                                   |
+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|[Richard Foreman. Christian Bale. ... , Nick Briggs. Sean Bean. ... , Kate Beckinsale. ... , Dirk Bogarde. ... , Michael Caine. ... , John Cleese. ... , Sacha Baron Cohen. ... , Photograph: YANNIS DRAKOULIDIS/2021.]|
+-------------------------------------------------------------------------------------------------------------------------------

In [ ]:
london_df.select(F.explode("answer_box.list").alias("actor_list ")).show(7, False)

+-------------------------------------+
|actor_list                           |
+-------------------------------------+
|Richard Foreman. Christian Bale. ... |
|Nick Briggs. Sean Bean. ...          |
|Kate Beckinsale. ...                 |
|Dirk Bogarde. ...                    |
|Michael Caine. ...                   |
|John Cleese. ...                     |
|Sacha Baron Cohen. ...               |
+-------------------------------------+
only showing top 7 rows



# Q9. Examine 'Orang juga bertanya'/'People Also Ask' in the London .json
For some reason, this London search result came from the Indonesian Google search engine. 'Orang juga bertanya' is Indonesian for 'People Also Ask'. 

Click through the HTML and find out 'Who is No 1 actor in the world?', then use code to extract the answer from the .json. Print the text of the answer to get full credit.

In [ ]:
snippet_Who_is_No_1_actor_in_the_world = london_df.select(F.col("related_questions.snippet").getItem(3).alias("Most Popular Actor"))
snippet_Who_is_No_1_actor_in_the_world.show(1, False)

+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|Most Popular Actor                                                                                                                                                                                                                                                                                               |
+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|American veteran actor Dwayne Johnson is the most popular actor in the worl

# Q10. Extract a list of the 10 websites listed on the London .json
Similar to what you did for Ukraine. These come from the organic results. Should include names like timeout.com, imdb.com, etc.

In [ ]:
london_websites = london_df.select(F.col("organic_results.displayed_link"))
london_websites.printSchema()

root
 |-- displayed_link: array (nullable = true)
 |    |-- element: string (containsNull = true)



In [ ]:
london_websites.select(F.explode("displayed_link").alias("London_Search_Websites")).show(10, False)


+--------------------------------------------------------+
|London_Search_Websites                                  |
+--------------------------------------------------------+
|https://www.imdb.com › name                             |
|https://www.mrdustbin.com › ...                         |
|https://www.glamourmagazine.co.uk › ... › Celebrity News|
|https://www.entoin.com › briti...                       |
|https://londranews.com › lond...                        |
|https://www.youtube.com › watch                         |
|https://www.youtube.com › w...                          |
|https://www.thegentlemansjournal.com › ...              |
+--------------------------------------------------------+



 Similar to question 7, I don't think there are 10 websites listed in Ukraine.json. I guess you counted an extra imdb website that comes under organic_results-->related_results-->displayed_link
 also my output list doesn't contain website "https://www.timeout.com" listed under answer_box-->link
 above code returns only websites under  organic_results-->displayed_link

# 🔴 Q11. Comments
Make three good bullets that describe what you learned in this assignment. Talk about how even JSON files that are semi-similar can still be difficult to work with if they are heterogenous! 

1. Both Ukraine.json and London.json have a key organic results which has more or less the same skeleton. May be that is the conventional standard of how a google search query information gets recorded in a json format. It is this format that helped me effortlessly solve the extra credit question.
2. I learned to retrieve/ extract data from specific hierarchy in json using the explode function.
3. I also learned about the works of three types in json namely, struct, array and string.



# 🔵 Extra Credit (5 pts)
This one is tough because Ukraine only has 9 links and London has 10 links!

Read both .json files at once using a wildcard, and make a dataframe with two rows (one for Ukraine and one for London) and 21 columns. The first column has the search query, and the next 10 columns are the (up to) 10 websites and the 10 columns after that are the 10 snippets. Good luck!

Max grade is 100 points on this assignment, if you still have a 105 score, it will be recoded as 100.

In [ ]:
DIRECTORY = '/content'

In [ ]:
two_searches_df = spark.read.json(os.path.join(DIRECTORY, '*.json'), multiLine=True)
two_searches_df.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|          answer_box|       inline_images|     organic_results|          pagination|   related_questions|    related_searches|  search_information|     search_metadata|   search_parameters|  serpapi_pagination|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                null|[[https://www.goo...|[[https://www.goo...|[1, https://www.g...|                null|[[https://www.goo...|[[[, 1,, All], [h...|[2023-02-18 19:45...|[desktop, google,...|[1, https://serpa...|
|[[[Time Out adala...|                null|[[,, [[IMDb adala...|[1, https://www.g...|[[, https://yougo...|[[https://www.goo...|[[[, 1,, Semua], ...|[202

In [ ]:
London_Ukraine_Combined = two_searches_df.select(
  "search_parameters.q",
  F.col("organic_results.displayed_link")[0].alias("Website 1"),
  F.col("organic_results.displayed_link")[1].alias("Website 2"),
  F.col("organic_results.displayed_link")[2].alias("Website 3"),
  F.col("organic_results.displayed_link")[3].alias("Website 4"),
  F.col("organic_results.displayed_link")[4].alias("Website 5"),
  F.col("organic_results.displayed_link")[5].alias("Website 6"),
  F.col("organic_results.displayed_link")[6].alias("Website 7"),
  F.col("organic_results.displayed_link")[7].alias("Website 8"),
  F.col("organic_results.snippet")[0].alias("Snippet 1"),
  F.col("organic_results.snippet")[1].alias("Snippet 2"),
  F.col("organic_results.snippet")[2].alias("Snippet 3"),
  F.col("organic_results.snippet")[3].alias("Snippet 4"),
  F.col("organic_results.snippet")[4].alias("Snippet 5"),
  F.col("organic_results.snippet")[5].alias("Snippet 6"),
  F.col("organic_results.snippet")[6].alias("Snippet 7"),
  F.col("organic_results.snippet")[7].alias("Snippet 8"),
)

London_Ukraine_Combined.show(2, False)

+---------------------------------------------------+------------------------------------------------------+--------------------------------------------------------+---------------------------------------------------------+--------------------------------------------------------+-----------------------------------------------+-------------------------------------+----------------------------------------------------+------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------

Continuing Q7 and Q9 instead of 10 websites and 10 snippets I have dispalyed 8 websites and 8 snippets.